In [16]:
from PIL import Image, ImageOps

import json
import cv2 
import os
import glob 
import re
from os import path

from tqdm import tqdm
import pickle
import pandas as pd
import numpy as np
import PIL.Image
import time
import pyarrow.parquet as pq
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline 

In [17]:
data_file = open("word_dict.pkl", "rb")
word_dict = pickle.load(data_file)

data_file.close()

In [18]:
words_dict=[]
for word in range(50):
    inp_file = str(word_dict[word])
    inp_file = inp_file.lstrip('0123456789_')
    #print(inp_file)
    words_dict.append(inp_file)

In [19]:
a_file = open("grapheme_dict.pkl", "rb")
graphemes_dict = pickle.load(a_file)
#print(grapheme_dict)

a_file.close()

In [20]:
label_file = open("labels.pkl", "rb")
labels = pickle.load(label_file)
#print(grapheme_dict)

label_file.close()


In [21]:
img_dir = "C:/Users/monsur/Desktop/tempF/graphemeImages"
data_path = os.path.join(img_dir,'*g')
files = glob.glob(data_path) 
file=[]
img_lbl=[]
for i in range(len(files)):
    file.append(files[i])
    inp_file=str(file[0])
    inp_file = inp_file.lstrip('C:/Users/monsur/Desktop/tempF/graphemeImages\ ')
    inp_file = inp_file.rstrip('.jpg')
    img_lbl.append(inp_file)
    
    file=[]
print(img_lbl)

['100', '1000', '1001', '1002', '1003', '1005', '1007', '1009', '101', '1011', '1012', '1014', '1015', '1017', '1018', '1019', '102', '1020', '1022', '1023', '1027', '1029', '103', '1030', '1032', '1035', '1036', '1037', '104', '1041', '1042', '1043', '1046', '1047', '1048', '105', '1050', '1051', '1052', '1053', '1055', '1057', '106', '1061', '1063', '1065', '1067', '1068', '1069', '107', '1070', '1072', '1074', '1075', '1077', '108', '1086', '1087', '1088', '1089', '109', '1090', '1093', '1094', '1096', '1098', '11', '110', '1104', '1105', '1109', '111', '1111', '1114', '1115', '1116', '1117', '1118', '1119', '112', '1121', '1123', '1124', '1125', '1128', '1129', '113', '1130', '1131', '1133', '1134', '1135', '1138', '1139', '114', '1140', '1141', '1142', '1145', '1146', '1148', '1149', '115', '1154', '1156', '1157', '1158', '1159', '116', '1161', '1164', '1167', '117', '1170', '1173', '1176', '1177', '118', '1181', '1182', '1183', '1185', '1187', '119', '1190', '1192', '1193', '1196

In [22]:

for idx in range(len(img_lbl)):
    img_name = img_lbl[idx]+'.jpg'
    img_opndir = path.join('C:/Users/monsur/Desktop/tempF/graphemeImages',img_name)
    img = Image.open(img_opndir)
    

In [23]:
print(graphemes_dict)

{'জা': 2, 'য়': 3, 'গা': 4, 'ই': 5, 'হা': 6, 'ড়া': 7, 'ভা': 8, 'ঙ্গা': 9, 'সার্': 10, 'ধ': 11, 'শ': 12, 'ত': 13, 'ম': 14, 'মে': 15, 'ল': 16, 'তে': 17, 'জি': 18, 'বু': 19, 'ব': 20, 'দৌঁ': 21, 'ড়ে': 22, 'ক্যা': 23, 'লে': 24, 'সে': 25, 'দ্ধ': 26, 'সর্': 27, 'কে': 28, '৷': 29, 'পা': 30, 'জে': 31, 'ঘূর্': 32, 'ণি': 33, 'ঝ': 34, 'ড়': 35, 'প্র': 36, 'ণ': 37, 'সৌ': 38, 'তা': 39, 'ন্ত্রি': 40, 'ক': 41, '২': 42, '১': 43, '৪': 44, '৩': 45, 'আ': 46, 'ভঁ': 47, 'রা': 48, 'ঙা': 49, 'মা': 50, 'টি': 51, 'য়া': 52, 'র্': 53, 'কু': 54, 'রী': 55, 'ঘো': 56, 'ষে': 57, 'র': 58, 'ট': 59, 'সা': 60, 'নো': 61, 'সি': 62, 'জ': 63, 'ন': 64, 'স': 65, 'রো': 66, 'দ': 67, 'ফি': 68, 'নে': 69, 'হী': 70, '০': 71, '৬': 72, '৯': 73, 'কি': 74, 'লা': 75, 'অ': 76, 'নু': 77, 'ষ্ঠা': 78, 'সূ': 79, 'চী': 80, 'চি': 81, 'ত্র': 82, 'চর্': 83, 'চা': 84, 'তো': 85, 'লো': 86, 'ম্পূর্': 87, 'রু': 88, 'পে': 89, 'আর্': 90, 'যা': 91, 'বর্': 92, 'দ্রী': 93, 'জ্ঞা': 94, 'ন্ডা': 95, 'রে': 96, 'রি': 97, 'দ্র': 98, 'দে': 99, 'ভি': 100, 'ন্ন':

In [24]:
print(labels[0])

[2, 3, 4, 3, 5]


In [25]:
print(len(img_lbl))
print(len(labels))


1064
799984


In [26]:
def crop_image(img):
    timg=img
    
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    th, threshed = cv2.threshold(img, 150, 255, cv2.THRESH_BINARY_INV)

    ## (2) Morph-op to remove noise
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (11,11))
    morphed = cv2.morphologyEx(threshed, cv2.MORPH_CLOSE, kernel)
    morphed = cv2.dilate(morphed,kernel,iterations = 2)

    ## (3) Find the max-area contour
    cnts = cv2.findContours(morphed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[-2]
    cnt = sorted(cnts, key=cv2.contourArea)[-1]

    ## (4) Crop and save it
    x,y,w,h = cv2.boundingRect(cnt)
    dst = img[y:y+h, x:x+w]
    tdst= timg[y:y+h, x:x+w]
    #timg = timg.crop((x, y, w, h))
    
    return tdst

In [27]:
def hconcat_resize_min(im_list, interpolation=cv2.INTER_CUBIC):
    h_min = min(im.shape[0] for im in im_list)
    im_list_resize = [cv2.resize(im, (int(im.shape[1] * h_min / im.shape[0]), h_min), interpolation=interpolation)
                      for im in im_list]
    return cv2.hconcat(im_list_resize)

In [28]:
#img_loc=[]
imgs=[]
index=0
for p in range(len(word_dict)):
    corrImg=False
    mergecount=0
    for q in range(len(labels[p])):
        #print("Original label: ",labels[p][q])
        #print(labels[p][q],"d")
        for i in range(len(img_lbl)):
            if(str(labels[p][q])==img_lbl[i]):
                #print("label Matched: ",img_lbl[i])
                img_name = img_lbl[i]+'.jpg'
                img_opndir = path.join('C:/Users/monsur/Desktop/tempF/graphemeImages',img_name)
                
                img = cv2.imread(img_opndir)
                
                img=crop_image(img)
                
                
                imgs.append(img)
                
                merge = hconcat_resize_min(imgs)
                #merge = cv2.hconcat(imgs)
               
                
                mergecount+=1
                break
    img=[]        
    imgs=[]
    if(mergecount==(len(labels[p]))):
        corrImg=True
        
    
    if(corrImg==True):
        index+=1
        im = Image.fromarray(merge)
        img_name= str(index)+"_"+word_dict[p]+".jpg"
        img_dir = path.join('C:/Users/monsur/Desktop/Tubappi/tempsetforcroppedconcatanated',img_name)
        im.save(img_dir)
        
        #print(img_name)
        #print(labels[p])
        
        imgs=[]
        img_name=[]
        img_opndir=[]
        #print(img[q])
    


In [29]:
print(img_lbl)

['100', '1000', '1001', '1002', '1003', '1005', '1007', '1009', '101', '1011', '1012', '1014', '1015', '1017', '1018', '1019', '102', '1020', '1022', '1023', '1027', '1029', '103', '1030', '1032', '1035', '1036', '1037', '104', '1041', '1042', '1043', '1046', '1047', '1048', '105', '1050', '1051', '1052', '1053', '1055', '1057', '106', '1061', '1063', '1065', '1067', '1068', '1069', '107', '1070', '1072', '1074', '1075', '1077', '108', '1086', '1087', '1088', '1089', '109', '1090', '1093', '1094', '1096', '1098', '11', '110', '1104', '1105', '1109', '111', '1111', '1114', '1115', '1116', '1117', '1118', '1119', '112', '1121', '1123', '1124', '1125', '1128', '1129', '113', '1130', '1131', '1133', '1134', '1135', '1138', '1139', '114', '1140', '1141', '1142', '1145', '1146', '1148', '1149', '115', '1154', '1156', '1157', '1158', '1159', '116', '1161', '1164', '1167', '117', '1170', '1173', '1176', '1177', '118', '1181', '1182', '1183', '1185', '1187', '119', '1190', '1192', '1193', '1196

In [30]:
print(len(words_dict))

50
